## Set-up

In [1]:
### Import Libraries
import pandas as pd
import numpy as np
import sklearn.metrics

classes = ["ABCA4", "BBS1", "BEST1", "CACNA1F", "CDH23", "CERKL", "CHM", "CNGA3", "CNGB3",
           "CRB1", "CRX", "CYP4V2", "EFEMP1", "EYS", "GUCY2D", "KCNV2", "MERTK", "MTTL1",
           "MYO7A", "NR2E3", "OPA1", "PDE6B", "PROML1", "PRPF31", "PRPF8", "PRPH2", "RDH12",
           "RHO", "RP1", "RP1L1", "RP2", "RPE65", "RPGR", "RS1", "TIMP3", "USH2A"]

# Test data
# N.B: This is only 1st appointment test data
df_all = pd.read_csv("results/predictions_all_test.csv")
df_external = df_all[df_all['hospital'] != "Moorfields"]
df_mf_test = df_all[df_all['hospital'] == "Moorfields"] 

# Moorfields test + CV
# This is all the moorfields predictions in subsequent appointments + Cross-validation predictions
df_mf = pd.read_csv("results/predictions_moorfields.csv") 
df_mf_cv = df_mf[df_mf['fold'] > -1] 

/tmp/ipykernel_4247/3638372330.py:13: DtypeWarning: Columns (10,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all = pd.read_csv("results/predictions_all_test.csv")


### Metrics and helper functions

In [2]:
# Grouby patient number
# and then group by another value and accregate predictions
def get_preds(df, group_on='file.path'):
    pandas_query = [ 'pred.'+cls for cls in classes ]
    truth, pred_scores, pat_ids = list(), list(), list()
    for pat_id, pat_entries in df.groupby('patient.number'):
        gene = pat_entries['gene'].values[0]
        gene_ind = classes.index(gene)
        for val, grouped_entries in pat_entries.groupby(group_on):
            pred_scores.append(grouped_entries[pandas_query].mean(axis=0))
            pat_ids.append(pat_id)
            truth.append(gene_ind)   
    return np.array(truth), np.array(pred_scores), pat_ids

# Metrics

def accuracy(truth, pred_scores):
    pred_class = np.argmax(pred_scores, axis=-1)
    correct = pred_class == truth
    return correct.mean()

def top_k(truth, pred_scores, k):
    rank = len(classes) - np.argwhere(np.argsort(pred_scores) == truth[:,np.newaxis])[:,1] # 'rank' of correct prediction
    return np.mean(rank <= k)

# The below section throws up a bunch of numpy warnings - these are handled later so redundant
import warnings
warnings.filterwarnings('ignore') 

def per_class_stats(truth, pred_scores, class_ind):
    class_true = truth == class_ind
    class_prediction = pred_scores[:, class_ind]
    class_predicted = pred_scores.argmax(axis=-1) == class_ind
    
    tp = class_true & class_predicted
    tn = ~ (class_true | class_predicted)
    
    precision = tp.sum() / class_predicted.sum()
    recall = tp.sum() / class_true.sum() # N.B: This is also Sensitivity
    specificity = tn.sum() / (1-class_true).sum()

    # Calculate Precision-Recall + ROC curves
    if class_true.any() and not class_true.all():
        pr_auc  = sklearn.metrics.average_precision_score(class_true, class_prediction, average="macro")
        #This is giving a different value to AUC of np PRC
        # Apparently average_precision_score calculates the mean Precision-Recall curve then takes the AUC,
        # while for mean AUC we calculate the AUPRC for each class then mean.
        roc_auc = sklearn.metrics.roc_auc_score(class_true, class_prediction)
    else:
        pr_auc, roc_auc = np.NAN, np.NAN
    
    return precision, recall, specificity, pr_auc, roc_auc

def mean_per_class_stats(truth, pred_scores):
    return np.nanmean([ per_class_stats(truth, pred_scores, i) for i in range(pred_scores.shape[-1]) ], axis=0)

def get_bootstrap_samples(metric_fn, truth, pred_scores, n_samples=1000, sample_size=None, n_workers=None):
    sample_size = sample_size if sample_size else len(truth)
    indices = np.random.choice(len(truth), size=(n_samples,sample_size))
    if n_workers:
        from multiprocessing import Pool
        with Pool(n_workers) as pool:
            result = list(pool.starmap(metric_fn, zip(truth[indices], pred_scores[indices])))
    else:
        result = list(map(metric_fn, truth[indices], pred_scores[indices]))
    return np.array(result)

# Results

In [9]:
def print_results(truth, pred_scores, bootstrapped=False):
    n_classes = pred_scores.shape[-1]
    
    # Accuracy and top-k
    pred_class = np.argmax(pred_scores, axis=-1)
    correct = pred_class == truth
    rank = n_classes - np.argwhere(np.argsort(pred_scores) == truth[:,np.newaxis])[:,1] # 'rank' of correct prediction

    print("Accuracy: {:2.1f}%".format(correct.mean() * 100))
    for i in [2,3,5,10,20,36]:
        print("top-{}: {:2.1f}%".format(i, np.mean(rank <= i) * 100))
    
    aucs = list()
    prcs_aucs = list()
    for i in range(n_classes):
        class_true = truth == i
        class_prediction = pred_scores[:,i]

        fpr, tpr, threshold = sklearn.metrics.roc_curve(class_true, class_prediction)
        aucs.append(sklearn.metrics.auc(fpr, tpr))
        pr, rec, threshold = sklearn.metrics.precision_recall_curve(class_true, class_prediction)
        prcs_aucs.append(sklearn.metrics.auc(rec, pr))
    print("Mean AUROC: {:0.3f}".format(np.nanmean(aucs)))
    print("Mean AUPRC: {:0.3f} (trapezoidal)".format(np.nanmean(prcs_aucs)))
    
    if bootstrapped:
        # Confidence intervals
        bootstrap_accuracy = get_bootstrap_samples(accuracy, truth, pred_scores, n_samples=10000)
        acc_mean = bootstrap_accuracy.mean(axis=0) * 100
        acc_upper = np.percentile(bootstrap_accuracy, 97.5) * 100
        acc_lower = np.percentile(bootstrap_accuracy, 2.5) * 100
        print("Bootstrapped Accuracy: {:2.1f}% ({:2.1f}-{:2.1f}%)".format(acc_mean,acc_lower,acc_upper))

        bootstrap_per_class = get_bootstrap_samples(mean_per_class_stats, truth, pred_scores, n_samples=1000, n_workers=100)
        # precision, recall, specificity, pr_auc, roc_auc
        pc_mean = bootstrap_per_class.mean(axis=0)
        pc_interval = np.percentile(bootstrap_per_class, [2.5,97.5], axis=0) 
        print("Bootstrapped AUROC: {:1.3f} ({:1.3f}-{:1.3f})".format(pc_mean[4], pc_interval[0,4], pc_interval[1,4]))
        print("Bootstrapped AUPRC: {:1.3f} ({:1.3f}-{:1.3f}) (average precision)".format(pc_mean[3], pc_interval[0,3], pc_interval[1,3]))
    
    print()

In [4]:
%%time

# Internal test set (Moorfields)
df = df_mf_test[df_mf_test['gene'].isin(classes)]
truth, pred_scores, pat_ids = get_preds(df, group_on='patient.number')

print("Internal test set")
print_results(truth, pred_scores, bootstrapped=True)

Internal test set
Accuracy: 66.7%
top-2: 75.4%
top-3: 80.7%
top-5: 85.6%
top-10: 94.7%
top-20: 98.9%
top-36: 100.0%
Mean AUROC: 0.935
Mean AUPRC: 0.564 (trapezoidal)
Bootstrapped Accuracy: 66.7% (61.0-72.3%)
Bootstrapped AUROC: 0.936 (0.919-0.955)
Bootstrapped AUPRC: 0.602 (0.528-0.678) (average precision)

CPU times: user 1.47 s, sys: 625 ms, total: 2.09 s
Wall time: 7.84 s


In [5]:
%%time

# External test set (Liverpool + Oxford + Bonn + Sao Paulo)
df = df_external[df_external['gene'].isin(classes)]
truth, pred_scores, pat_ids = get_preds(df, group_on='patient.number')

print("External test set")
print_results(truth, pred_scores, bootstrapped=True)

External test set
Accuracy: 65.3%
top-2: 74.2%
top-3: 80.9%
top-5: 86.4%
top-10: 93.2%
top-20: 98.3%
top-36: 100.0%
Mean AUROC: 0.930
Mean AUPRC: 0.442 (trapezoidal)
Bootstrapped Accuracy: 65.2% (59.3-71.2%)
Bootstrapped AUROC: 0.928 (0.902-0.951)
Bootstrapped AUPRC: 0.540 (0.465-0.612) (average precision)

CPU times: user 1.32 s, sys: 717 ms, total: 2.04 s
Wall time: 8.15 s


In [6]:
%%time

# Combined
df = df_all[df_all['gene'].isin(classes)]
truth, pred_scores, pat_ids = get_preds(df, group_on='patient.number')

print("All test data")
print_results(truth, pred_scores, bootstrapped=True)

All test data
Accuracy: 66.0%
top-2: 74.8%
top-3: 80.8%
top-5: 86.0%
top-10: 94.0%
top-20: 98.6%
top-36: 100.0%
Mean AUROC: 0.932
Mean AUPRC: 0.449 (trapezoidal)
Bootstrapped Accuracy: 66.0% (61.8-70.0%)
Bootstrapped AUROC: 0.933 (0.917-0.947)
Bootstrapped AUPRC: 0.504 (0.441-0.572) (average precision)

CPU times: user 1.74 s, sys: 3.88 s, total: 5.62 s
Wall time: 14.1 s


### By individual network

In [13]:
model_names =[{'BAF': 'models_baf/17012021-173755-InceptionV3-100e-128bs-0.0001lr.h5',
               'OCT': 'models_oct/18012021-095554-InceptionV3-100e-128bs-0.0001lr.h5',
               'IR':  'models_ir/18012021-203445-InceptionV3-100e-128bs-0.0001lr.h5'},
              {'BAF': 'models_baf/17012021-202904-InceptionV3-100e-128bs-0.0001lr.h5',
               'OCT': 'models_oct/18012021-120608-InceptionV3-100e-128bs-0.0001lr.h5',
               'IR':  'models_ir/19012021-010048-InceptionV3-100e-128bs-0.0001lr.h5'},
              {'BAF': 'models_baf/17012021-232108-InceptionV3-100e-128bs-0.0001lr.h5',
               'OCT': 'models_oct/18012021-141228-InceptionV3-100e-128bs-0.0001lr.h5',
               'IR':  'models_ir/19012021-051220-InceptionV3-100e-128bs-0.0001lr.h5'},
              {'BAF': 'models_baf/18012021-023016-InceptionV3-100e-128bs-0.0001lr.h5',
               'OCT': 'models_oct/18012021-161956-InceptionV3-100e-128bs-0.0001lr.h5', 
               'IR':  'models_ir/19012021-091140-InceptionV3-100e-128bs-0.0001lr.h5'}, 
              {'BAF': 'models_baf/18012021-052425-InceptionV3-100e-128bs-0.0001lr.h5', 
               'OCT': 'models_oct/18012021-182559-InceptionV3-100e-128bs-0.0001lr.h5', 
               'IR':  'models_ir/19012021-131130-InceptionV3-100e-128bs-0.0001lr.h5'}]

In [ ]:
%%time
df_test = df_mf_test[df_mf_test['gene'].isin(classes)]

print("Test set results:")

for modality in ["BAF", "IR", "OCT" ]:
    #df_modality = df_test[df_test["modality"] == modality]
    for fold in range(5):
        
        df = df_test[df_test['pred.model'] == model_names[fold][modality]]
        truth, pred_scores, pat_ids = get_preds(df, group_on='file.path') # filter of filepath for individual results

        print(modality, "model", str(fold+1))
        print(model_names[fold][modality])
        print_results(truth, pred_scores)

## Restricted to certain classes

In [5]:
def restrict_classes(labels, preds, subclasses):
    subclass_idxs = [ classes.index(cls) for cls in subclasses ]
    subclass_newidxs = [ subclasses.index(cls) for cls in subclasses ]
    idx_map = dict(zip(subclass_idxs, subclass_newidxs))
    rws = np.isin(labels, subclass_idxs)
    preds = preds[rws][:, subclass_idxs]
    labels = np.vectorize(idx_map.get)(labels[rws])
    return labels, preds

In [11]:
df = df_mf_test[df_mf_test['gene'].isin(classes)]
truth, pred_scores, pat_ids = get_preds(df, group_on='patient.number')

truth, pred_scores = restrict_classes(truth, pred_scores, ["ABCA4", "USH2A"])

print_results(truth, pred_scores, bootstrapped=True)

0.8740157480314961
Accuracy: 87.4%
top-2: 99.2%
top-3: 100.0%
top-5: 100.0%
top-10: 100.0%
top-20: 100.0%
top-36: 100.0%
Mean AUROC: 0.940
Mean AUPRC: 0.871 (trapezoidal)
Bootstrapped Accuracy: 87.4% (81.1-92.9%)
Bootstrapped AUROC: 0.940 (0.899-0.973)
Bootstrapped AUPRC: 0.875 (0.801-0.938) (average precision)

